## **STEP 1: Loading dataset**

In [1]:
from google.colab import files
uploaded = files.upload() 

Saving sonar.csv to sonar.csv


## Step 2. Baseline Neural Network Model Performance

In [0]:
import numpy
import pandas
import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# load dataset
dataframe = pandas.read_csv("sonar.csv", header=None)
dataset = dataframe.values
# split into input (X) and output (Y) variables
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]

# Encoding
le = LabelEncoder()
encoded_Y = le.fit(Y)
encoded_Y = le.transform(Y)
train_data = X
train_targets = encoded_Y

In [0]:
def create_baseline():
  model = Sequential()
  model.add(Dense(60, activation = 'relu', input_shape=(60,)))
  model.add(Dense(1, activation = 'sigmoid'))
  
  model.compile(loss = 'binary_crossentropy',
                optimizer = 'adam',
                metrics = ['accuracy'])
  return model

In [4]:
# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, encoded_Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Results: 83.71% (6.13%)


## **Step 3 : Re-Run The Baseline Model With Data Preparation**

In [5]:
# evaluate baseline model with standardized dataset
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Standardized: 85.59% (7.46%)


## **Step 4.1: Evaluate a Smaller Network**

In [0]:
def create_smaller():
  model = Sequential()
  model.add(Dense(30, activation = 'relu', input_shape=(60,)))
  model.add(Dense(1, activation = 'sigmoid'))
  
  model.compile(loss = 'binary_crossentropy',
                optimizer = 'adam',
                metrics = ['accuracy'])
  return model

In [7]:
# evaluate baseline model with standardized dataset
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_smaller, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Smaller: 85.06% (7.61%)


## **Step 4.2. Evaluate a Larger Network**

In [0]:
def create_larger():
  model = Sequential()
  model.add(Dense(60, activation = 'relu', input_shape=(60,)))
  model.add(Dense(30, activation = 'relu', input_shape=(60,)))
  model.add(Dense(1, activation = 'sigmoid'))
  
  model.compile(loss = 'binary_crossentropy',
                optimizer = 'adam',
                metrics = ['accuracy'])
  return model

In [9]:
# evaluate baseline model with standardized dataset
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))



Larger: 84.61% (6.36%)


## **Step 5: Really Scaling up: developing a model that overfits & Step 6: Tuning the Model**

In [0]:
def create_larger():
  model = Sequential()
  model.add(Dense(64, activation = 'relu', input_shape=(60,)))
  model.add(Dense(50, activation = 'relu'))
  model.add(Dense(32, activation = 'relu'))
  model.add(Dense(15, activation = 'relu'))
  model.add(Dense(5, activation = 'relu'))
  model.add(Dense(1, activation = 'sigmoid'))
  
  model.compile(loss = 'binary_crossentropy',
                optimizer = 'adam',
                metrics = ['accuracy'])
  return model

In [11]:
# evaluate baseline model with standardized dataset
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=20, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Model results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))



Model results: 86.98% (9.28%)


## **Step 7: Rewriting the code using the Keras Functional API**

In [0]:
def create_func_api_model():
  inputs = keras.Input(shape=(60,))
  x = layers.Dense(50, activation = 'relu')(inputs)
  x = layers.Dense(32, activation = 'relu')(x)
  x = layers.Dense(15, activation = 'relu')(x)
  x = layers.Dense(5, activation = 'relu')(x)
  outputs = layers.Dense(1, activation = 'sigmoid')(x)
  
  model = keras.Model(inputs, outputs)
  model.compile(loss = 'binary_crossentropy',
                optimizer = 'adam',
                metrics = ['accuracy'])
  return model

In [19]:
# evaluate baseline model with standardized dataset
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_func_api_model, epochs=300, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Function API model results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))



Function API model results: 83.13% (5.94%)


## **Step 8: Rewriting the code by doing Model Subclassing**

In [0]:
# TODO

## **Step 9: Rewriting the code without using scikit-learn**

In [0]:
def baseline_model():
  model = Sequential()
  model.add(Dense(64, activation = 'relu', input_shape=(60,)))
  model.add(Dense(50, activation = 'relu'))
  model.add(Dense(32, activation = 'relu'))
  model.add(Dense(15, activation = 'relu'))
  model.add(Dense(5, activation = 'relu'))
  model.add(Dense(1, activation = 'sigmoid'))
  
  model.compile(loss = 'binary_crossentropy',
                optimizer = 'adam',
                metrics = ['accuracy'])
  return model

In [10]:
import numpy as np

k=5
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []

num_epochs = 500
batch_size_val = num_val_samples // k
all_mae_histories = []
for i in range(k):
    print('processing fold #', i)
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
    
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)
    
    model = baseline_model()
    history = model.fit(partial_train_data, partial_train_targets,
                    validation_data=(val_data, val_targets),
                    epochs=num_epochs, batch_size=batch_size_val, verbose=0)
    print("Training Accuracy:", history.history['acc'][num_epochs-1])
    print("Validation Accuracy:", history.history['val_acc'][num_epochs-1])


processing fold # 0
Training Accuracy: 1.0
Validation Accuracy: 0.43902439024390244
processing fold # 1
Training Accuracy: 1.0
Validation Accuracy: 0.6585365853658537
processing fold # 2
Training Accuracy: 1.0
Validation Accuracy: 0.4634146341463415
processing fold # 3
Training Accuracy: 1.0
Validation Accuracy: 0.5609756097560976
processing fold # 4
Training Accuracy: 1.0
Validation Accuracy: 0.6341463414634146
